# II. Ensembles

В задачах нужно корректно реализовать функции, чтобы проходили тесты. 

## 1. Bootstrap.

### Алгоритм Bootstrap 
* Равномерно возьмем из выборки $N$ объектов **с возвращением**. То есть мы хотим сгенерировать псевдовыборку, в которой могут повторятся элементы из исходной выборки. 

* Обозначим новую выборку через $X_1$. Повторяя процедуру $B$ раз, сгенерируем $M$ подвыборок $X_1, \dots, X_B$. 

* Посчитаем статистику T от каждой выборки $(T(X_1), \ldots, T(X_n))$

* Найдем итоговую статистику $T(X) = \frac{1}{B}\sum^{B}_{i}T(X_i)$

На вход массив чисел $X$ и число бутстрепных выборок $B$. Необходимо реализовать свой бутстреп и найти матожидание и стандартную ошибку у бутстрепных выборок.


### TASK

In [1]:
import numpy as np
from scipy.stats import sem # ищет SE среднего

def get_stats(X: np.array, B:int)->tuple:
    '''
        .∧＿∧ 
        ( ･ω･｡)つ━☆・*。 
        ⊂  ノ    ・゜+. 
        しーＪ   °。+ *´¨) 
                .· ´¸.·*´¨) 
                (¸.·´ (¸.·'* ☆  <YOUR CODE>
    '''
    N = len(X)
    mean, SE = 0, 0
    
    for i in range(B):
        X_k = np.random.choice(X,N)
        m, s = X_k.mean(), sem(X_k)
        mean +=m
        SE+=s
        
    mean=mean/B
    SE=SE/B

    return mean, SE

### Open tests

In [2]:
######################################################
X = np.array([37,43,38,36,17,40,40,45,41,84])
B = 100000

mean, se = get_stats(X, B)

assert np.abs(mean - 42.1) < 0.05
assert np.abs(se - 4.56) < 0.03
######################################################

# 2. Bagging

Необходимо реализовать свой небольшой беггинг на деревьях заданной грубины

* бустингом сделать несколько выборок $X_1, \ldots, X_B$
* обучить на этих выборках алгоритмы: $a_1(\cdot), \ldots, a_B(\cdot)$

Получить результат беггинга как:
$$a(x) = \frac{1}{B}\sum_{b=1}^{B}a_b(x)$$


# TASK

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor as DTR
import random

def bagging(X_train, y_train, X_test, boot_count, depth):
    estimators = np.array([DTR(max_depth=depth) for _ in range(boot_count)])
    
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:YOUR CODE HERE
    
    y_pred = np.array( estimators[0].fit(X_train, y_train).predict(X_test))  

    for i in range(1, boot_count):
        
        n = len(y_train)
        
        ind = [random.randint(0, n-1) for x in range(0, n)]
        
        X = np.array([X_train[ind[0]]] )
        y = np.array([y_train[ind[0]]] )
        
        for j in ind[1:]:
            X = np.append(X, [X_train[j]], axis = 0)
            y = np.append(y, y_train[j])
        
        y_pred+= np.array( estimators[i].fit(X, y).predict(X_test))
    
    y_pred = y_pred/boot_count
    
    return y_pred

In [7]:
import numpy as np
from numpy.testing import assert_array_equal, assert_array_almost_equal, assert_equal, assert_almost_equal
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

######################################################

X_train = np.array([[0, 0], [1, 1], [5, 5], [8, 8], [10, 10]])
y_train = np.array([0, 1, 5, 8, 10])
X_test  = np.array([[4, 4], [6, 6]])
y_test  = np.array([4, 6])

B = 1000

y_pred = bagging(X_train, y_train, X_test, boot_count=B, depth=3)
print(y_pred)
assert_array_almost_equal(y_pred, np.array([4, 6]), decimal=0)
######################################################

[3.647 6.053]


In [257]:
from sklearn.datasets import load_boston

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

X, y = load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=123,
                                                    shuffle=True)

y_pred = bagging(X_train, y_train, X_test, boot_count=200, depth=10)

y_dt_pred = DecisionTreeRegressor().fit(X_train, y_train).predict(X_test)
y = RandomForestRegressor().fit(X_train, y_train).predict(X_test)

print(mean_squared_error(y, y_test))
print(mean_squared_error(y_dt_pred, y_test))
print(mean_squared_error(y_pred, y_test))
assert mean_squared_error(y_pred, y_test) < 15 

16.14051578947369
17.647697368421053
14.306911699155107


### 3. X-regression

Необходимо найти наилучшие параметры для XGBRegression, обучить модель и вернуть ее. Данные берутся из папки data.

Сам гридсерч или нативное исследование необходимо делать вне функции обработки, чтобы не получить TL.

### TASK

In [255]:
from xgboost.sklearn import XGBRegressor

def xreg(X_train: np.array, y_train:np.array) -> XGBRegressor:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    model = XGBRegressor(objective ='reg:squarederror', n_estimators = 6, max_depth = 7, random_state=17).fit(X_train, y_train)
    return model

### OPEN TESTS

In [253]:
import xgboost 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.read_csv('Financial Distress.csv') 

X = df.drop('Financial Distress', axis=1)
y = df['Financial Distress']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)


xgb1 = XGBRegressor(objective ='reg:squarederror')

n_estimators = range(1, 10)
max_depth = range(1, 10)

parameters = {'max_depth': max_depth, 'n_estimators': n_estimators}

xgb_grid = GridSearchCV(xgb1, parameters, cv = 5, n_jobs = -1, verbose=True)

xgb_grid.fit(X_train, y_train, w)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 398 out of 405 | elapsed:   21.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:   22.1s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:squarederror',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': range(1, 10),
                         'n_estimators': range(1, 10)},
          

In [256]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

t1 = time.time()
xgb_model = xreg(X_train, y_train)
t2 = time.time()
assert t2 - t1 < 10


y_pred = xgb_model.predict(X_test)

print(MSE(y_pred, y_test))

assert type(xgb_model) == xgboost.sklearn.XGBRegressor
assert MSE(y_pred, y_test) < 3
print('Well Done')

0.22721506335199831
{'max_depth': 7, 'n_estimators': 6}
2.264570030358505
Well Done


## 4. CatFeatures

Обучите модель классификации катбуста на предложенных данных и верните обученную модель. 

Воспользуйтесь встроенной обработкой категориальных признаков. Не забудьте обработать Nan значения.

### TASK

In [161]:
import catboost
def catfeatures(df: pd.DataFrame):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    categorical = [ 'Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']
    
    df_tr = df.copy()
    df_tr['dep_delayed_15min'] = df_tr['dep_delayed_15min'].replace('N', 0)
    df_tr['dep_delayed_15min'] = df_tr[['dep_delayed_15min']].replace('Y', 1)
    
    df_tr = df_tr.drop(['dep_delayed_15min'], axis = 1).copy()
    
    train_data = catboost.Pool(data = df_tr, label=train_label, cat_features = categorical)
    
    model = catboost.CatBoostClassifier(iterations = 15, depth = 7)
    model.fit(train_data)
    return model

### TESTS

In [166]:
import time

#%%time
df = pd.read_csv('flight_delays_train.csv')


df_train = df[:7000]

t1 = time.time()
model = catfeatures(df_train)
t2 = time.time()

assert t2 - t1 < 10
assert type(model) == catboost.CatBoostClassifier

Learning rate set to 0.5
0:	learn: 0.5435703	total: 7ms	remaining: 98ms
1:	learn: 0.4953870	total: 12.8ms	remaining: 82.9ms
2:	learn: 0.4765525	total: 17.9ms	remaining: 71.5ms
3:	learn: 0.4673303	total: 25.3ms	remaining: 69.6ms
4:	learn: 0.4612785	total: 30.6ms	remaining: 61.2ms
5:	learn: 0.4584288	total: 35.7ms	remaining: 53.6ms
6:	learn: 0.4543716	total: 41.2ms	remaining: 47.1ms
7:	learn: 0.4536558	total: 43.6ms	remaining: 38.2ms
8:	learn: 0.4520514	total: 49.3ms	remaining: 32.8ms
9:	learn: 0.4498994	total: 54.3ms	remaining: 27.2ms
10:	learn: 0.4458257	total: 60.2ms	remaining: 21.9ms
11:	learn: 0.4448928	total: 64.9ms	remaining: 16.2ms
12:	learn: 0.4420538	total: 70.6ms	remaining: 10.9ms
13:	learn: 0.4406757	total: 75.5ms	remaining: 5.39ms
14:	learn: 0.4389093	total: 81ms	remaining: 0us


In [165]:
from sklearn.metrics import accuracy_score
    
df_test = pd.read_csv('flight_catfeature_test.csv')
df_test = df_test.drop('Unnamed: 0', axis=1)

X_test = df_test.drop('dep_delayed_15min',axis=1)
y_test = df_test['dep_delayed_15min']

y_pred = model.predict(X_test)

print('Well Done')

Well Done


Пусть мы хотим бустить регрессию со стандартной функцией потерь $MSE$:

$$\mathcal{L}(a, x,y) = (a(x_i) - y_i)^2$$

Необходимо найти через взятие производных:

1. Константный вектор $[f_0]_{i=1}^{N}$
$$f_0(x) = \arg\min_{ c\in \mathbb{R}} \sum_{i=1}^n \mathcal{L}(c, y_i)$$ 

2. Градиенты функции потерь
$$g_{i}^{t} = -\Big[\frac{\partial \mathcal{L}(f_t, x_i, y_i)}{\partial f_t(x_i)}\Big]_{i=1}^N$$

3. Коэффициенты при композиции 
$$\alpha_{t + 1} = \arg\min_\alpha \sum_{i=1}^N \mathcal{L}(f_{t}(x_i) + \alpha b_{t+1}(x_i), y_i)$$

### TASK

In [13]:
import numpy as np

def init(y_i: np.array) -> float:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:
    # (c - y1)^2 + (c - y2)^2 +...+ (c - yn)^2 = f Возьмем производную и приравняем к нулю, тогда c - это среднее от y.
    f_0 = y_i.mean()
    return f_0

def grad(a: np.array, y: np.array) -> np.array:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    # Тут по формуле для L расписываем
    g = -2*(a-y)
    return g

def alpha(f :np.array, b: np.array, y: np.array) -> float:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    alpha = ( (y*b).sum() - (f*b).sum())/( (b*b).sum())
    # print(alpha)
    return alpha

### TESTS

In [15]:
y = np.array([1, 2, 3])
f = np.array([2, 2, 2])
b = np.array([0, 2, 4])

f_0 = init(y)
g = grad(f,y)
al = alpha(f,b,y)


assert np.abs(f_0 - 2.0)   < 1e-9
assert_array_almost_equal(g, np.array([-2, 0, 2]))
assert np.abs(al - (0.2)) < 1e-9
# Мне кажется здесь alpha должен быть все-таки без минуса (если посчитать производную, там нет минуса).

In [16]:
######################################################
y = np.arange(20)
f = np.ones(20) * 10
b = np.arange(20) - 1

f_0 = init(y)
g = grad(f,y)
al = alpha(f,b,y)


assert np.abs(f_0 - 9.5)   < 1e-2
assert_array_almost_equal(g, np.arange(-20,20, 2))
assert np.abs(al - (0.2748)) < 1e-2
print('Well Done!')

Well Done!


## 6. GradientBoosting

Реализуйте градиентный бустинг на решающих деревьях для регрессии с логгированием.Верните модель, которая будет хранить в себе `n_estimatos` обученных деревьев и коэффициенты, чтобы с их помощью потом найти результат предсказания.

Также необходимо реализовать логгирование в течение обучения.
* `self.estimators` - лист c деревьями
* `self.alpha` - лист с коэффициентами альфа
* `self.f_list` - лист со значениями комбинаций алгоритма $f_T(x_i) = f_0(x_i) + \sum_{t=1}^{T}\alpha_tb_t(x_i)$
* `self.g_list` - лист с векторами градиентов на каждой итерации $g_{i}^{t} = -\Big[\frac{\partial \mathcal{L}(f_t, x_i, y_i)}{\partial f_t(x_i)}\Big]_{i=1}^N$
* `self.b_list` - лист со значениями базового обучаемого дерева на тренировачной выборке на каждой итерации 

Примечания:
* Обрывать алгоритм не нужно, необходимо обучить все деревья.
* Начальный константный вектор из $f_0$ логгировать не нужно, однако не забудьте его добавить в `predict` c нужным количеством объектов!

### TASK

In [41]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.metrics import mean_squared_error

class MyGradBoost():
    def __init__(self, n_estimators=10, max_depth=3):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.estimators_ = np.array([DTR(max_depth=self.max_depth) for _ in range(n_estimators)])
        self.alpha = []
        self.f_list = []
        self.b_list = []
        self.g_list = []
        
    def fit(self, X_train: np.array, y_train: np.array): 
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        
        T = self.n_estimators
        f_0 = init(y_train)
        
        self.f_list.append( [f_0]*len(y_train) )
        self.g_list.append( grad( self.f_list[0], y_train) ) 
        self.b_list.append(1)
        self.alpha.append(1)
        
        for t in range(1, T+1):
            
            self.estimators_[t-1] = self.estimators_[t-1].fit(X_train, self.g_list[t-1] )
            self.b_list.append( self.estimators_[t-1].predict(X_train) )
            self.alpha.append( alpha(self.f_list[t-1], self.b_list[t],  y_train  ) )  
            self.f_list.append( self.f_list[t-1]+self.alpha[t]*self.b_list[t] )
            self.g_list.append( grad(self.f_list[t], y_train) )
          
        return self
        
    def predict(self, X_test) -> np.array:
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        
        T = self.n_estimators        
        f_0 = self.f_list[0][0]
        y_pred = np.array([f_0]*len(X_test))
        for t in range(1, T+1):
            y_pred += np.array( self.alpha[t]*self.estimators_[t-1].predict(X_test) ) 
        return y_pred
    
    def score(self, X_test, y_test)-> np.array:
        return mean_squared_error(self.predict(X_test), y_test)

### TESTS

In [51]:
n_estimators = 2
max_depth=3
X_train = np.array([[0], [1], [2], [3], [4]])
y_train = np.array([0, 2, 4, 2, 0])
X_test  = np.array([[1.2], [2.3]])
y_test  = np.array([2.2, 3.7])

model = MyGradBoost(n_estimators=n_estimators, max_depth=max_depth).fit(X_train, y_train)
print(model.score(X_test, y_test))
assert model.score(X_test, y_test) < 0.2
######################################################

0.06499999999999999


In [54]:
n_train, n_test, noise = 150, 1000, 0.1
# Generate data
def f(x):
    x = x.ravel()
    return np.exp(-x ** 2) + 1.5 * np.exp(-(x - 2) ** 2)

def generate(n_samples, noise):
    X = np.random.rand(n_samples) * 10 - 5
    X = np.sort(X).ravel()
    y = np.exp(-X ** 2) + 1.5 * np.exp(-(X - 2) ** 2)\
        + np.random.normal(0.0, noise, n_samples)
    X = X.reshape((n_samples, 1))

    return X, y

X_train, y_train = generate(n_samples=n_train, noise=noise)
X_test, y_test = generate(n_samples=n_test, noise=noise)


model = MyGradBoost().fit(X_train, y_train)


assert model.score(X_test, y_test) < 0.025


model = MyGradBoost(n_estimators=100, 
                    max_depth=1).fit(X_train, y_train)

print(model.score(X_test, y_test))
assert model.score(X_test, y_test) < 0.017
print('Well Done')

0.014152544896247073
Well Done


# Самопроверка

In [55]:
import matplotlib.pyplot as plt
def predict_and_plot(model, X_test, y_test, title):
    y_predict = model.predict(X_test)

    plt.plot(X_test, f(X_test), "b")
    plt.scatter(X_train, y_train, c="b", s=20)
    plt.plot(X_test, y_predict, "g", lw=2)
    plt.xlim([-5, 5])
    plt.title("{} Loss: {:2f}".format(title, model.score(X_test, y_test)))
    plt.grid()



model = MyGradBoost(n_estimators=30, 
                    max_depth=1).fit(X_train, y_train)

ind =  [1,3,5,10,15,30]

# GradientBoostingRegressor
plt.plot(X_test, f(X_test), "b")
plt.scatter(X_train, y_train, c="b", s=20)
n_est = [1,3,5,10,15,30]
f = np.array(model.f_list)
for i, n in enumerate(n_est):
    colors = ['g', 'r', 'c', 'm', 'y', 'k']
    plt.plot(X_train, f[n-1], color=colors[i], label="tree count={}".format(n))

plt.xlim([-5, 5])   
plt.legend()
plt.grid()
plt.show()

<Figure size 640x480 with 1 Axes>